# Clustering Crypto

In [2]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [17]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head()

Unnamed: 0   CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
0         42    42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
1        365    365Coin       X11       True   PoW/PoS              NaN   
2        404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
3        611  SixEleven   SHA-256       True       PoW              NaN   
4        808        808   SHA-256       True   PoW/PoS     0.000000e+00   

  TotalCoinSupply  
0              42  
1      2300000000  
2       532000000  
3          611000  
4               0

In [18]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]                            
crypto_df.head()

Unnamed: 0   CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
0         42    42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
1        365    365Coin       X11       True   PoW/PoS              NaN   
2        404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
3        611  SixEleven   SHA-256       True       PoW              NaN   
4        808        808   SHA-256       True   PoW/PoS     0.000000e+00   

  TotalCoinSupply  
0              42  
1      2300000000  
2       532000000  
3          611000  
4               0

In [19]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].isnull().values.any()

False

In [20]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

Unnamed: 0   CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
0         42    42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
1        365    365Coin       X11   PoW/PoS              NaN      2300000000
2        404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
3        611  SixEleven   SHA-256       PoW              NaN          611000
4        808        808   SHA-256   PoW/PoS     0.000000e+00               0

In [21]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [22]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]                            
crypto_df.head()

Unnamed: 0   CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
0         42    42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
2        404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
5       1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
7        BTC    Bitcoin   SHA-256       PoW     1.792718e+07        21000000
8        ETH   Ethereum    Ethash       PoW     1.076842e+08               0

In [24]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = pd.DataFrame(crypto_df['CoinName'])
crypto_names.head()

CoinName
0    42 Coin
2    404Coin
5  EliteCoin
7    Bitcoin
8   Ethereum

In [25]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df.head()

Unnamed: 0 Algorithm ProofType  TotalCoinsMined TotalCoinSupply
0         42    Scrypt   PoW/PoS     4.199995e+01              42
2        404    Scrypt   PoW/PoS     1.055185e+09       532000000
5       1337       X13   PoW/PoS     2.927942e+10    314159265359
7        BTC   SHA-256       PoW     1.792718e+07        21000000
8        ETH    Ethash       PoW     1.076842e+08               0

In [26]:
crypto_df = crypto_df.set_index('Unnamed: 0')
crypto_df.head()

Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                     
42            Scrypt   PoW/PoS     4.199995e+01              42
404           Scrypt   PoW/PoS     1.055185e+09       532000000
1337             X13   PoW/PoS     2.927942e+10    314159265359
BTC          SHA-256       PoW     1.792718e+07        21000000
ETH           Ethash       PoW     1.076842e+08               0

In [28]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(data=crypto_df, columns=['Algorithm','ProofType'])
X.head()

TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
Unnamed: 0                                                                      
42             4.199995e+01              42                                 0   
404            1.055185e+09       532000000                                 0   
1337           2.927942e+10    314159265359                                 0   
BTC            1.792718e+07        21000000                                 0   
ETH            1.076842e+08               0                                 0   

            Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  \
Unnamed: 0                                                         
42                      0                  0                   0   
404                     0                  0                   0   
1337                    0                  0                   0   
BTC                     0                  0                   0   
ETH                     0                  0                   0   

            Algorithm_Blake  Algorithm_Blake2S  Algorithm_Blake2b  \
Unnamed: 0                                                          
42                        0                  0                  0   
404                       0                  0                  0   
1337                      0                  0                  0   
BTC                       0                  0                  0   
ETH                       0                  0                  0   

            Algorithm_C11  ...  ProofType_PoW/PoS  ProofType_PoW/PoS   \
Unnamed: 0                 ...                                          
42                      0  ...                  1                   0   
404                     0  ...                  1                   0   
1337                    0  ...                  1                   0   
BTC                     0  ...                  0                   0   
ETH                     0  ...                  0                   0   

            ProofType_PoW/PoW  ProofType_PoW/nPoS  ProofType_Pos  \
Unnamed: 0                                                         
42                          0                   0              0   
404                         0                   0              0   
1337                        0                   0              0   
BTC                         0                   0              0   
ETH                         0                   0              0   

            ProofType_Proof of Authority  ProofType_Proof of Trust  \
Unnamed: 0                                                           
42                                     0                         0   
404                                    0                         0   
1337                                   0                         0   
BTC                                    0                         0   
ETH                                    0                         0   

            ProofType_TPoS  ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
Unnamed: 0                                                                      
42                       0                               0                   0  
404                      0                               0                   0  
1337                     0                               0                   0  
BTC                      0                               0                   0  
ETH                      0                               0                   0  

[5 rows x 98 columns]

In [29]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [30]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)
crypto_pca

array([[-0.33518947,  1.03065133, -0.59406007],
       [-0.31852355,  1.03080946, -0.5945003 ],
       [ 2.30435153,  1.62716779, -0.68765498],
       ...,
       [ 0.32225658, -2.294787  ,  0.38790062],
       [-0.15977967, -2.14023147,  0.50274445],
       [-0.28804051,  0.84282251, -0.3033524 ]])

In [32]:
# Create a DataFrame with the three principal components.
df_crypto_pca = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index = crypto_df.index
)
df_crypto_pca.head()

PC 1      PC 2      PC 3
Unnamed: 0                              
42         -0.335189  1.030651 -0.594060
404        -0.318524  1.030809 -0.594500
1337        2.304352  1.627168 -0.687655
BTC        -0.142757 -1.315761  0.234351
ETH        -0.150276 -1.999008  0.349050

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [33]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [34]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)
print(predictions)

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 1 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 3 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [39]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, df_crypto_pca], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_names['CoinName'].values

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


Algorithm ProofType  TotalCoinsMined TotalCoinSupply  \
Unnamed: 0                                                              
42                  Scrypt   PoW/PoS     4.199995e+01              42   
404                 Scrypt   PoW/PoS     1.055185e+09       532000000   
1337                   X13   PoW/PoS     2.927942e+10    314159265359   
BTC                SHA-256       PoW     1.792718e+07        21000000   
ETH                 Ethash       PoW     1.076842e+08               0   
LTC                 Scrypt       PoW     6.303924e+07        84000000   
DASH                   X11   PoW/PoS     9.031294e+06        22000000   
XMR         CryptoNight-V7       PoW     1.720114e+07               0   
ETC                 Ethash       PoW     1.133597e+08       210000000   
ZEC               Equihash       PoW     7.383056e+06        21000000   

                PC 1      PC 2      PC 3          CoinName  Class  
Unnamed: 0                                                         
42         -0.335189  1.030651 -0.594060           42 Coin      0  
404        -0.318524  1.030809 -0.594500           404Coin      0  
1337        2.304352  1.627168 -0.687655         EliteCoin      0  
BTC        -0.142757 -1.315761  0.234351           Bitcoin      3  
ETH        -0.150276 -1.999008  0.349050          Ethereum      3  
LTC        -0.164500 -1.111559 -0.025708          Litecoin      3  
DASH       -0.396719  1.231675 -0.518981              Dash      0  
XMR        -0.144307 -2.210467  0.399837            Monero      3  
ETC        -0.148718 -1.999109  0.349025  Ethereum Classic      3  
ZEC        -0.159779 -2.140231  0.502744             ZCash      3

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [42]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_data=['CoinName','Algorithm'],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [43]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [44]:
# Print the total number of tradable cryptocurrencies.
total_cryptos = clustered_df['CoinName'].count()
print(f"There are {total_cryptos} total tradable cryptocurrencies.")

There are 532 total tradable cryptocurrencies.


In [45]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_scaled = MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
print(clustered_scaled)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [47]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(clustered_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df['Class']
plot_df.head(10)

TotalCoinSupply  TotalCoinsMined          CoinName  Class
Unnamed: 0                                                           
42             4.200000e-11         0.000000           42 Coin      0
404            5.320000e-04         0.001066           404Coin      0
1337           3.141593e-01         0.029576         EliteCoin      0
BTC            2.100000e-05         0.000018           Bitcoin      3
ETH            0.000000e+00         0.000109          Ethereum      3
LTC            8.400000e-05         0.000064          Litecoin      3
DASH           2.200000e-05         0.000009              Dash      0
XMR            0.000000e+00         0.000017            Monero      3
ETC            2.100000e-04         0.000115  Ethereum Classic      3
ZEC            2.100000e-05         0.000007             ZCash      3

In [50]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols=['CoinName'])

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)